In [911]:
# Dependencies
import pandas as pd
import warnings
import numpy as np
warnings.simplefilter(action='ignore', category=FutureWarning)
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans
import matplotlib.pyplot as plt


In [912]:
# Read in crypto data csv
df = pd.read_csv('crypto_data.csv')
df.head()

,Unnamed: 0,CoinName,Algorithm,IsTrading,ProofType,TotalCoinsMined,TotalCoinSupply
0,42,42 Coin,Scrypt,True,PoW/PoS,4.199995e+01,42
1,365,365Coin,X11,True,PoW/PoS,NaN,2300000000
2,404,404Coin,Scrypt,True,PoW/PoS,1.055185e+09,532000000
3,611,SixEleven,SHA-256,True,PoW,NaN,611000
4,808,808,SHA-256,True,PoW/PoS,0.000000e+00,0


In [913]:
# Check # of rows and columns
df.shape

(1252, 7)

In [914]:
# Get indexes where IsTrading is False
indexNames = df[df['IsTrading'] == False].index

# Delete False rows from dataframe
df.drop(indexNames, inplace=True)


In [915]:
# Recheck # of rows and columns to make sure False values dropped
df.shape

(1144, 7)

In [916]:
# Drop IsTrading column
df = df.drop(columns='IsTrading')


In [917]:
# Preview dataframe
df.head()

,Unnamed: 0,CoinName,Algorithm,ProofType,TotalCoinsMined,TotalCoinSupply
0,42,42 Coin,Scrypt,PoW/PoS,4.199995e+01,42
1,365,365Coin,X11,PoW/PoS,NaN,2300000000
2,404,404Coin,Scrypt,PoW/PoS,1.055185e+09,532000000
3,611,SixEleven,SHA-256,PoW,NaN,611000
4,808,808,SHA-256,PoW/PoS,0.000000e+00,0


In [918]:
# Drop rows with null values
df = df.dropna(how='any', axis=0)

In [919]:
# Any drops?
df.shape

(685, 6)

In [920]:
# Filter for cryptocurrencies that have been mined
df = df.loc[df['TotalCoinsMined'] !=0]


In [921]:
# Any drops?
df.head()

,Unnamed: 0,CoinName,Algorithm,ProofType,TotalCoinsMined,TotalCoinSupply
0,42,42 Coin,Scrypt,PoW/PoS,4.199995e+01,42
2,404,404Coin,Scrypt,PoW/PoS,1.055185e+09,532000000
5,1337,EliteCoin,X13,PoW/PoS,2.927942e+10,314159265359
7,BTC,Bitcoin,SHA-256,PoW,1.792718e+07,21000000
8,ETH,Ethereum,Ethash,PoW,1.076842e+08,0


In [922]:
# Drop CoinName column
df = df.drop(columns='CoinName')

In [923]:
# Preview dataframe
df.head()

,Unnamed: 0,Algorithm,ProofType,TotalCoinsMined,TotalCoinSupply
0,42,Scrypt,PoW/PoS,4.199995e+01,42
2,404,Scrypt,PoW/PoS,1.055185e+09,532000000
5,1337,X13,PoW/PoS,2.927942e+10,314159265359
7,BTC,SHA-256,PoW,1.792718e+07,21000000
8,ETH,Ethash,PoW,1.076842e+08,0


In [924]:
# Drop Unnamed: 0 column
df = df.drop(columns='Unnamed: 0')

In [925]:
df.head()

,Algorithm,ProofType,TotalCoinsMined,TotalCoinSupply
0,Scrypt,PoW/PoS,4.199995e+01,42
2,Scrypt,PoW/PoS,1.055185e+09,532000000
5,X13,PoW/PoS,2.927942e+10,314159265359
7,SHA-256,PoW,1.792718e+07,21000000
8,Ethash,PoW,1.076842e+08,0


In [926]:
# Convert Algorithm and ProofType to numerical data by creating dummy variables
dummy_df = pd.get_dummies(df, columns=['Algorithm', 'ProofType'])
dummy_df.head()

,TotalCoinsMined,TotalCoinSupply,Algorithm_1GB AES Pattern Search,Algorithm_536,Algorithm_Argon2d,Algorithm_BLAKE256,Algorithm_Blake,Algorithm_Blake2S,Algorithm_Blake2b,Algorithm_C11,...,ProofType_PoW/PoS,ProofType_PoW/PoS,ProofType_PoW/PoW,ProofType_PoW/nPoS,ProofType_Pos,ProofType_Proof of Authority,ProofType_Proof of Trust,ProofType_TPoS,ProofType_Zero-Knowledge Proof,ProofType_dPoW/PoW
0,4.199995e+01,42,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
2,1.055185e+09,532000000,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
5,2.927942e+10,314159265359,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
7,1.792718e+07,21000000,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
8,1.076842e+08,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [927]:
# Check data types
dummy_df.dtypes

TotalCoinsMined                     float64
TotalCoinSupply                      object
Algorithm_1GB AES Pattern Search      uint8
Algorithm_536                         uint8
Algorithm_Argon2d                     uint8
                                     ...   
ProofType_Proof of Authority          uint8
ProofType_Proof of Trust              uint8
ProofType_TPoS                        uint8
ProofType_Zero-Knowledge Proof        uint8
ProofType_dPoW/PoW                    uint8
Length: 100, dtype: object

In [928]:
# Convert object data types to float and verify
dummy_df['TotalCoinSupply'] = pd.to_numeric(dummy_df.TotalCoinSupply, errors='coerce')
dummy_df.dtypes


TotalCoinsMined                     float64
TotalCoinSupply                     float64
Algorithm_1GB AES Pattern Search      uint8
Algorithm_536                         uint8
Algorithm_Argon2d                     uint8
                                     ...   
ProofType_Proof of Authority          uint8
ProofType_Proof of Trust              uint8
ProofType_TPoS                        uint8
ProofType_Zero-Knowledge Proof        uint8
ProofType_dPoW/PoW                    uint8
Length: 100, dtype: object

In [929]:
# Standardize data with StandardScaler
scaler = StandardScaler()
scaled_data = scaler.fit_transform(dummy_df[['TotalCoinsMined', 'TotalCoinSupply']])


In [930]:
# Create new df from scaled data
scaled_df = pd.DataFrame(scaled_data, columns=dummy_df.columns[0:2])
scaled_df.to_csv('scaled.csv')

In [931]:
# Drop old totalcoinsmined and totalcoinsupply columns
dummy_df = dummy_df.drop(columns=['TotalCoinsMined', 'TotalCoinSupply']).reset_index()
dummy_df.shape


(533, 99)

In [932]:
# Add scaled columns to dummy_df
dummy_df['TotalCoinsMined'] = scaled_df['TotalCoinsMined']
dummy_df['TotalCoinSupply'] = scaled_df['TotalCoinSupply']
dummy_df.head()

,index,Algorithm_1GB AES Pattern Search,Algorithm_536,Algorithm_Argon2d,Algorithm_BLAKE256,Algorithm_Blake,Algorithm_Blake2S,Algorithm_Blake2b,Algorithm_C11,Algorithm_Cloverhash,...,ProofType_PoW/PoW,ProofType_PoW/nPoS,ProofType_Pos,ProofType_Proof of Authority,ProofType_Proof of Trust,ProofType_TPoS,ProofType_Zero-Knowledge Proof,ProofType_dPoW/PoW,TotalCoinsMined,TotalCoinSupply
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,-0.116748,-0.152865
1,2,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,-0.093589,-0.144996
2,5,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0.525872,4.493764
3,7,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,-0.116354,-0.152554
4,8,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,-0.114384,-0.152865


In [933]:
# Drop old index
dummy_df = dummy_df.drop(columns='index')
dummy_df.head()

,Algorithm_1GB AES Pattern Search,Algorithm_536,Algorithm_Argon2d,Algorithm_BLAKE256,Algorithm_Blake,Algorithm_Blake2S,Algorithm_Blake2b,Algorithm_C11,Algorithm_Cloverhash,Algorithm_Counterparty,...,ProofType_PoW/PoW,ProofType_PoW/nPoS,ProofType_Pos,ProofType_Proof of Authority,ProofType_Proof of Trust,ProofType_TPoS,ProofType_Zero-Knowledge Proof,ProofType_dPoW/PoW,TotalCoinsMined,TotalCoinSupply
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,-0.116748,-0.152865
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,-0.093589,-0.144996
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0.525872,4.493764
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,-0.116354,-0.152554
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,-0.114384,-0.152865


In [934]:
# Initialize PCA model
pca = PCA(n_components=12)

# Get two principal components for the data
data_pca = pca.fit_transform(dummy_df)

In [935]:
df_pca = pd.DataFrame(data=data_pca)
df_pca.head()

,0,1,2,3,4,5,6,7,8,9,10,11
0,-0.208958,-0.801427,0.172224,-0.673560,-0.027430,-0.027398,0.002287,-0.017059,-0.047586,-0.023039,-0.007695,0.015708
1,-0.187051,-0.803933,0.161796,-0.674183,-0.028180,-0.027263,0.002391,-0.018045,-0.047734,-0.023073,-0.007599,0.015808
2,3.533384,-0.301848,2.918907,0.408402,-0.165250,-0.258806,-0.276661,-0.158009,0.746460,0.361772,-0.233944,-0.070549
3,-0.179922,0.694159,-0.166384,0.239206,-0.278880,-0.613804,0.625230,-0.014071,0.027751,0.009416,-0.026391,-0.026452
4,-0.173076,0.688440,-0.155140,0.131556,-0.212781,-0.104400,-0.241486,-0.000390,0.113146,-0.125557,0.058427,-0.129463


In [936]:
# Fetch the explained variance
pca.explained_variance_ratio_.sum()

0.9039491205806693